In [ ]:
import numpy as np
import scipy.interpolate
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from typing import Union, Literal, Callable, Tuple, List
import warnings

strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    label_from_attrs,
    adjust_lightness_array,
    plot_one_one,
    handler_map_alpha,
    save_figure,
    add_second_x_axis,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean
from sdm_eurec4a import RepositoryPath

from sdm_eurec4a import slurm_cluster
import dask.distributed

default_colors = set_custom_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

RepoPaths = RepositoryPath("levante")

data_dir = RepoPaths.CLEO_data_dir / Path("output_v4.1")
fig_dir = RepoPaths.fig_dir / Path("paper-v4.1")
fig_dir.mkdir(exist_ok=True, parents=False)

In [ ]:
try:
    _, _ = client, cluster
except Exception:
    print("Starting dask cluster")
    port = 8287
    client, cluster = slurm_cluster.init_dask_slurm_cluster(
        scale=1,
        processes=16,
        memory="32GB",
        walltime="00:15:00",
        scheduler_options={"dashboard_address": f":{port}"},
    )
    display(client)

Starting dask cluster


In [ ]:
def evaporation_mass2evaporation_energy(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    L_v = 2.2257e3

    return x * L_v


def evaporation_mass2evaporation_precipitation(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    rho_water = 1000  # kg / m^3
    # L_v = 2.2257e3

    return x * 3600 / 1000 * rho_water


def evaporation_precipitation2evaporation_mass(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    rho_water = 1000  # kg / m^3
    L_v = 2257e3  # J / kg
    return x / 3600 * 1000 / rho_water


def evaporation_precipitation2evaporation_energy(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:
    rho_water = 1000  # kg / m^3
    L_v = 2257e3  # J / kg

    # x in mm/h

    # output in W/m^2

    x = x / 1000 * rho_water / 3600  # kg/m^2/s

    return x * L_v  # W/m^2


def evap_rate_mass2evap_rate_energy(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:

    L_v = 2257e3

    # x in g/m^3/s
    return x * L_v  # mW m^-3


def evap_rate_energy2evap_rate_mass(
    x: Union[xr.DataArray, np.ndarray]
) -> Union[xr.DataArray, np.ndarray]:

    L_v = 2257e3  # J / kg = W s / kg

    # x in mW m^-3
    return x / L_v  # g/m^3/s

In [ ]:
eulerian_data_path = lambda data_dir, microphysics: data_dir / Path(
    f"{microphysics}/combined/eulerian_dataset_combined.nc"
)
conservation_data_path = lambda data_dir, microphysics: data_dir / Path(
    f"{microphysics}/combined/conservation_dataset_combined.nc"
)


# mean_radius_data_path = lambda microphysics: data_dir / Path(
#     f"{microphysics}/combined/mean_radius_combined.nc"
# )

In [ ]:
microphysics = (
    "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)

compare_microphysics = (
    # "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
)

labels = {
    "null_microphysics": "Null",
    "condensation": "Evap only",
    "collision_condensation": "Coll",
    "coalbure_condensation_small": "CoalBuRe few",
    "coalbure_condensation_large": "CoalBuRe many",
}

colors = {
    "null_microphysics": "grey",
    "condensation": "purple",
    "collision_condensation": "blue",
    "coalbure_condensation_small": "red",
    "coalbure_condensation_large": "orange",
}

dark_colors_list = adjust_lightness_array(colors.values(), 0.75)
dark_colors = dict([(key, str(dark_colors_list[i])) for i, key in enumerate(colors.keys())])

light_colors_list = adjust_lightness_array(colors.values(), 1.25)
light_colors = dict([(key, str(light_colors_list[i])) for i, key in enumerate(colors.keys())])

markers = {
    "null_microphysics": "$*$",
    "condensation": "1",
    "collision_condensation": "x",
    "coalbure_condensation_small": "+",
    "coalbure_condensation_large": "2",
}
dark_colors

{'null_microphysics': '#606060',
 'condensation': '#600060',
 'collision_condensation': '#0000bf',
 'coalbure_condensation_small': '#bf0000',
 'coalbure_condensation_large': '#bf7c00'}

## Data loading

In [ ]:
for mp in microphysics:
    try:
        print(mp)
        xr.open_dataset(conservation_data_path(data_dir, mp))
    except ValueError:
        print(f"Missing conservation {mp}")
        continue
    try:
        print(mp)
        xr.open_dataset(eulerian_data_path(data_dir, mp))
    except ValueError:
        print(f"Missing eulerian {mp}")
        continue

null_microphysics
null_microphysics
condensation
condensation
collision_condensation
collision_condensation
coalbure_condensation_small
coalbure_condensation_small
coalbure_condensation_large
coalbure_condensation_large


In [ ]:
vars_to_use = [
    "air_temperature",
    "gridbox",
    "gridbox_bottom",
    "gridbox_coord3",
    "gridbox_top",
    "gridbox_volume",
    "gridbox_coord3_norm",
    "liquid_water_content",
    "mass_radius_mean",
    "mass_radius_std",
    "mass_represented_temporal_mean",
    "massdelta_condensation",
    "max_gridbox",
    "precipitation",
    "pressure",
    "radius_bins",
    "relative_humidity",
    "small_mass_radius_mean",
    "small_mass_radius_std",
    "small_xi_radius_mean",
    "small_xi_radius_std",
    "specific_mass_vapour",
    "surface_area",
    "xi_radius_mean",
    "xi_radius_std",
    "xi_temporal_mean",
]

### Merge of individual datasets

In [ ]:
time_slice = slice(1500, 3490)
# chunks = {'cloud_id' : 2}
chunks = {}

l = []
for microphysic in microphysics:
    print(microphysic)
    ds_euler = xr.open_dataset(eulerian_data_path(data_dir, microphysic))
    ds_euler = ds_euler[vars_to_use]
    ds_euler = ds_euler.sel(time=time_slice).mean("time", keep_attrs=True)
    ds_euler["surface_area"] = ds_euler["surface_area"].mean()
    ds_euler["surface_area"].attrs["units"] = "m^2"
    ds_euler["surface_area"].attrs["long_name"] = "Surface area"

    ds_euler["max_gridbox"] = ds_euler["max_gridbox"].fillna(ds_euler["gridbox"].max())

    # from kg per dT to g per h per m^2
    ds_conser = xr.open_dataset(conservation_data_path(data_dir, microphysic), chunks=chunks)
    ds_conser = ds_conser.sel(time=time_slice).mean("time", keep_attrs=True)

    # ds_mean_radius = xr.open_dataset(mean_radius_data_path(microphysic), chunks=chunks)

    ds_single = xr.merge([ds_conser, ds_euler])
    ds_single = ds_single.expand_dims(microphysics=(microphysic,))
    l.append(ds_single)
ds = xr.concat(l, dim="microphysics")
ds.chunk(chunks)

ds["max_gridbox"] = ds["max_gridbox"].fillna(ds["gridbox"].max())

ds["liquid_water_content"] = 1e3 * ds["liquid_water_content"]
ds["liquid_water_content"].attrs["units"] = "g/m^3"
ds["liquid_water_content"].attrs["long_name"] = "Rain water content"

ds["cloud_liquid_water_content"] = ds["liquid_water_content"].sel(gridbox=ds["max_gridbox"])
ds["cloud_liquid_water_content"].attrs["long_name"] = "Cloud rain water content"

# create variables for the mean radius of the distributions

# ds["cloud_xi_radius_mean"] = ds["xi_radius_mean"].sel(gridbox=ds["max_gridbox"])
# ds["cloud_xi_radius_mean"].attrs["long_name"] = "Cloud mean radius"
# ds["cloud_xi_radius_mean"].attrs["units"] = "µm"

# ds["small_cloud_xi_radius_mean"] = ds["small_xi_radius_mean"].sel(gridbox=ds["max_gridbox"])
# ds["small_cloud_xi_radius_mean"].attrs["long_name"] = "Cloud mean radius"
# ds["small_cloud_xi_radius_mean"].attrs["units"] = "µm"

ds["cloud_mass_radius_mean"] = ds["mass_radius_mean"].sel(gridbox=ds["max_gridbox"])
ds["cloud_mass_radius_mean"].attrs["long_name"] = "Cloud mean mass radius"
ds["cloud_mass_radius_mean"].attrs["units"] = "µm"

# ds["small_cloud_mass_radius_mean"] = ds["small_mass_radius_mean"].sel(gridbox=ds["max_gridbox"])
# ds["small_cloud_mass_radius_mean"].attrs["long_name"] = "Cloud mean mass radius"
# ds["small_cloud_mass_radius_mean"].attrs["units"] = "µm"

ds["source"].attrs["long_name"] = "Evaporation"
ds["inflow"].attrs["long_name"] = "Cloud Base Precipitation Flux"
ds["outflow"].attrs["long_name"] = "Surface Precipitation Flux"

# from kg/m^3/s

# # to mg/m^3/h
# ds['evaporation_rate']  = - ds['massdelta_condensation'] * 1e6 * 3600
# ds['evaporation_rate'].attrs['units'] = 'mg/m^3/h'
# ds['evaporation_rate'].attrs['long_name'] = 'Evaporation rate'

# to mm/m/h
rho_water = 1000  # kg / m^3
ds["evaporation_rate"] = -1e3 / rho_water * ds["massdelta_condensation"] * 3600
ds["evaporation_rate"].attrs["units"] = r"mm \, h^{-1} \, m^{-1}"
ds["evaporation_rate"].attrs["long_name"] = "Evaporation rate"

# from kg / m^3 / s
# to W / m^3
vapourization_heat = 2257e3  # J / kg
rho_water = 1000  # kg / m^3
ds["evaporation_rate_energy"] = ds["massdelta_condensation"] * vapourization_heat  # J/s / m^3 = W / m^3
ds["evaporation_rate_energy"] = 1e3 * ds["evaporation_rate_energy"]  # mW / m^3
ds["evaporation_rate_energy"].attrs["units"] = r"mW \, m^{-3}"
ds["evaporation_rate_energy"].attrs["long_name"] = "Evaporation rate"


for var, new_var in zip(
    ["source", "inflow", "outflow", "reservoir_change"],
    [
        "source_precipitation",
        "inflow_precipitation",
        "outflow_precipitation",
        "reservoir_change_precipitation",
    ],
):
    attrs = ds[var].attrs.copy()
    # from  kg per dT per domain area
    # dT = 2s

    # # to    g per h per m^2
    # ds[var] = ds[var] / 2 * 3600 / ds['surface_area'] * 1e6
    # ds[var].attrs.update(attrs)
    # ds[var].attrs['units'] = 'mg/m^2/h'

    # to    mm / h
    rho_water = 1000  # kg / m^3
    ds[new_var] = ds[var] / 2 * 3600 / ds["surface_area"]  # kg / m^2 / h
    ds[new_var] = 1e3 * ds[new_var] / rho_water  # mm / h
    ds[new_var].attrs.update(attrs)
    ds[new_var].attrs["units"] = r"mm \, h^{-1}"

for var, new_var in zip(
    ["source", "inflow", "outflow", "reservoir_change"],
    ["source_energy", "inflow_energy", "outflow_energy", "reservoir_change_energy"],
):
    attrs = ds[var].attrs.copy()
    # from  kg per dT per domain area
    # dT = 2s

    # # to    g per h per m^2
    # ds[var] = ds[var] / 2 * 3600 / ds['surface_area'] * 1e6
    # ds[var].attrs.update(attrs)
    # ds[var].attrs['units'] = 'mg/m^2/h'

    # to    mm / h
    vapourization_heat = 2257e3  # J / kg
    rho_water = 1000  # kg / m^3
    ds[new_var] = ds[var] / 2 / ds["surface_area"]  # kg / m^2 / s
    ds[new_var] = ds[new_var] * vapourization_heat  # J/s = W / m^2
    ds[new_var].attrs.update(attrs)
    ds[new_var].attrs["units"] = r"W \, m^{-2}"


ds["evaporation_fraction"] = -100 * ds["source"] / ds["inflow"]
ds["evaporation_fraction"].attrs["units"] = "\\%"
ds["evaporation_fraction"].attrs["long_name"] = "Evaporation fraction"

sorted_cloud_id = ds["cloud_id"].sortby(ds["evaporation_rate"].max(["gridbox", "microphysics"]))
ds = ds.load()
ds = ds.sel(cloud_id=sorted_cloud_id[:-1])
# ds_diff = ds - ds.sel(microphysics = 'condensation')

null_microphysics


condensation
collision_condensation
coalbure_condensation_small
coalbure_condensation_large


In [ ]:
# ds = ds.sel(cloud_id=sorted_cloud_id[:-2])
ds = ds.sortby("cloud_id")
ds_diff = ds - ds.sel(microphysics="condensation")

for var in ds_diff.data_vars:
    ds_diff[var].attrs.update(ds[var].attrs)
    try:
        ds_diff[var].attrs["long_name"] = f"Difference {ds[var].attrs['long_name']}"
    except KeyError:
        print(f"Missing long_name for {var}")

ds_relative_diff = (ds / ds.sel(microphysics="condensation") - 1) * 100

for var in ds_diff.data_vars:
    ds_relative_diff[var].attrs.update(ds[var].attrs)
    try:
        ds_relative_diff[var].attrs["long_name"] = f"Relative difference {ds[var].attrs['long_name']}"
        ds_relative_diff[var].attrs["units"] = r"\%"
    except KeyError:
        print(f"Missing long_name for {var}")

Missing long_name for air_temperature
Missing long_name for mass_represented_temporal_mean
Missing long_name for pressure
Missing long_name for specific_mass_vapour
Missing long_name for xi_temporal_mean
Missing long_name for air_temperature
Missing long_name for mass_represented_temporal_mean
Missing long_name for pressure
Missing long_name for specific_mass_vapour
Missing long_name for xi_temporal_mean


### Interpolation Dataset

In [ ]:
def interpolate_omit_nan(
    x: np.ndarray,
    xp: np.ndarray,
    fp: np.ndarray,
    method: Literal["linear", "cubic"] = "linear",
    kwargs: dict = {},
) -> np.ndarray:
    """
    Interpolate a 1-D function, omitting NaN values in the input arrays.
    Parameters
    ----------
    x : np.ndarray
        The x-coordinates at which to evaluate the interpolated values.
    xp : np.ndarray
        The x-coordinates of the data points, must be increasing.
    fp : np.ndarray
        The y-coordinates of the data points, same length as `xp`.
    method : str, optional
        The interpolation method to use. Supported methods are 'linear' and 'cubic'.
        Default is 'linear'.
    Returns
    -------
    np.ndarray
        The interpolated values, with NaNs where interpolation could not be performed.
    Notes
    -----
    - If there are fewer than 3 valid (non-NaN) points in `xp` and `fp`, the function
        will return an array of NaNs.
    - For 'cubic' interpolation, `scipy.interpolate.CubicSpline` is used.
    Examples
    --------
    >>> import numpy as np
    >>> x = np.array([0, 1, 2, 3, 4])
    >>> xp = np.array([0, 1, 2, np.nan, 4])
    >>> fp = np.array([0, 1, 4, np.nan, 16])
    >>> interpolate_omit_nan(x, xp, fp)
    array([ 0.,  1.,  4., nan, 16.])
    """
    # Function implementation here

    # create mask for valid points
    mask = np.isfinite(xp) & np.isfinite(fp)

    # only run interpolation if there are at least 3 valid points
    if np.sum(mask) > 2:

        # apply mask to remove nans from the data
        xp = xp[mask]
        fp = fp[mask]

        # interpolate linear, using the np.interp function
        if method == "linear":
            result = np.interp(x=x, xp=xp, fp=fp, left=np.nan, right=np.nan, **kwargs)
        # interpolate cubic, using the scipy.interpolate.CubicSpline function
        elif method == "cubic":
            result = scipy.interpolate.interp1d(xp, fp, kind="cubic", **kwargs)(x)
    # otherwise, return array of NaNs
    else:
        result = np.full_like(x, np.nan)

    return result


# Pytest test routines
def test_interpolate_linear():
    x = np.array([0, 1, 2, 3, 4])
    xp = np.array([0, 1, 2, np.nan, 4])
    fp = np.array([0, 1, 4, np.nan, 16])
    expected = np.array([0.0, 1.0, 4.0, np.nan, 16.0])
    result = interpolate_omit_nan(x, xp, fp, method="linear")
    np.testing.assert_array_equal(result, expected)


def test_interpolate_cubic():
    x = np.array([0, 1, 2, 3, 4])
    xp = np.array([0, 1, 2, 3, 4])
    fp = np.array([0, 1, 8, 27, 64])
    expected = np.array([0.0, 1.0, 8.0, 27.0, 64.0])
    result = interpolate_omit_nan(x, xp, fp, method="cubic")
    np.testing.assert_array_equal(result, expected)


def test_interpolate_insufficient_points():
    x = np.array([0, 1, 2, 3, 4])
    xp = np.array([0, np.nan, np.nan, np.nan, 4])
    fp = np.array([0, np.nan, np.nan, np.nan, 16])
    expected = np.full_like(x, np.nan)
    result = interpolate_omit_nan(x, xp, fp, method="linear")
    np.testing.assert_array_equal(result, expected)

In [ ]:
mapped_dim = ds["gridbox_coord3_norm"]
new_dim = np.linspace(-0.1, 1.1, 102)
new_dim = xr.DataArray(
    new_dim,
    coords={"normalized_gridbox_coord3": new_dim},
    attrs=dict(
        long_name="Normalized gridbox coordinate",
        units="",
        description="Normalized gridbox coordinate by the maximum gridbox coordinate",
    ),
)
new_dim.name = "normalized_gridbox_coord3"
new_dim_attrs = new_dim.attrs.copy()


def interpolate_dataset(ds: xr.Dataset, mapped_dim: xr.DataArray, new_dim: xr.DataArray) -> xr.Dataset:
    ds_interpolated = xr.Dataset()

    for variable in ds.data_vars:
        dims = ds[variable].dims
        attrs = ds[variable].attrs.copy()
        data = ds[variable]

        # only apply for those where the old dimension is a dimension
        if set(mapped_dim.dims).issubset(set(dims)):

            da = xr.apply_ufunc(
                interpolate_omit_nan,
                new_dim,
                mapped_dim,
                data,
                input_core_dims=[["normalized_gridbox_coord3"], ["gridbox"], ["gridbox"]],
                output_core_dims=[["normalized_gridbox_coord3"]],
                vectorize=True,
                kwargs={"method": "linear"},
            )
            da.attrs.update(attrs)

            ds_interpolated[variable] = da
        else:
            pass
    ds_interpolated[new_dim.name].attrs = new_dim_attrs
    return ds_interpolated


ds_interpolated = interpolate_dataset(ds, mapped_dim, new_dim)
ds_diff_relative_interpolated = interpolate_dataset(ds_relative_diff, mapped_dim, new_dim)

### Functions to convert precipitation to Energy values

# Visulization

## Precipitation

In [ ]:
fig = plt.figure()

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

x = ds["cloud_liquid_water_content"]
y = ds["inflow_precipitation"]


for mp in microphysics:
    for _ax in axs:
        _ax.scatter(
            x.sel(microphysics=mp),
            y.sel(microphysics=mp),
            color=colors[mp],
            alpha=0.5,
            marker=markers[mp],
            label=labels[mp],
        )
for _ax in axs:
    _ax.plot(
        x,
        y,
        color="grey",
        linewidth=0.2,
        alpha=0.2,
        zorder=0,
    )

    _ax.legend(loc="upper left")
axs[1].set_yscale("log")
axs[1].set_xscale("log")

for _ax in axs:

    _ax.set_xlabel(label_from_attrs(x))
    _ax.set_ylabel(label_from_attrs(y))
fig.suptitle(f"Relation of {x.attrs['long_name']} and {y.attrs['long_name']}")

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"precipitation-{x.name}_vs_{y.name}")

<Figure size 800x450 with 0 Axes>

In [ ]:
fig = plt.figure()

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

x = ds["cloud_liquid_water_content"]
y = -ds["outflow_precipitation"]


for mp in microphysics:
    for _ax in axs:
        _ax.scatter(
            x.sel(microphysics=mp),
            y.sel(microphysics=mp),
            color=colors[mp],
            alpha=0.5,
            marker=markers[mp],
            label=labels[mp],
        )
for _ax in axs:
    _ax.plot(
        x,
        y,
        color="grey",
        linewidth=0.2,
        alpha=0.2,
        zorder=0,
    )

    _ax.legend(loc="upper left")
axs[1].set_yscale("log")
axs[1].set_xscale("log")

for _ax in axs:

    _ax.set_xlabel(label_from_attrs(x))
    _ax.set_ylabel(label_from_attrs(y))
fig.suptitle(f"Relation of {x.attrs['long_name']} and {y.attrs['long_name']}")

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"precipitation-{x.name}_vs_{y.name}")

<Figure size 800x450 with 0 Axes>

### Precipitation statistics

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["inflow_precipitation"]

for mp in ["condensation"]:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )
    # ax.fill_betweenx(
    #     [0, 1],
    #     m - sem,
    #     m + sem,
    #     color=colors[mp],
    #     alpha=0.1,
    # )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 10.4, 0.2),
        label=label,
        # density=True,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 10.4, 0.2),
        histtype="step",
        # density=True,
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())

add_second_x_axis(
    ax,
    new_xticks_func=lambda x: evaporation_precipitation2evaporation_energy(x).astype(int),
    xlabel=label_from_attrs(ds["outflow_energy"]),
    position="bottom",
    offset_position=("axes", -0.15),
)
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")
fig.tight_layout()
save_figure(fig, fig_dir / f"histogram_{x.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = -ds["outflow_precipitation"]

for mp in ["condensation"]:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )
    # ax.fill_betweenx(
    #     [0, 1],
    #     m - sem,
    #     m + sem,
    #     color=colors[mp],
    #     alpha=0.1,
    # )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 10.4, 0.2),
        label=label,
        # density=True,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 10.4, 0.2),
        histtype="step",
        # density=True,
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())

add_second_x_axis(
    ax,
    new_xticks_func=lambda x: evaporation_precipitation2evaporation_energy(x).astype(int),
    xlabel=label_from_attrs(ds["outflow_energy"]),
    position="bottom",
    offset_position=("axes", -0.15),
)
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"histogram_{x.name}")

## Evaporation only setup

### Evaporation profiles

In [ ]:
mp = "condensation"

x = -ds["evaporation_rate_energy"]
y = ds["gridbox_coord3"]

fig, ax = plt.subplots()
ax.plot(
    x.sel(microphysics=mp).T,
    y.sel(microphysics=mp).T,
    color=colors[mp],
    alpha=0.1,
)

add_second_x_axis(
    ax,
    new_xticks_func=lambda _x: (1e3 * 3600 * evap_rate_energy2evap_rate_mass(_x)).astype(int),
    xlabel=label_from_attrs(ds["evaporation_rate_energy"], return_units=False)
    + r" $\left[mg \, m^{-3} h^{-1}\right]$",
    position="bottom",
    offset_position=("axes", -0.15),
)

# plt.xscale('log')
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}_vs_{y.name}_{mp}")

In [ ]:
mp = "condensation"

x = -ds_interpolated["evaporation_rate_energy"]
y = ds_interpolated["normalized_gridbox_coord3"]

fig, ax = plt.subplots()
ax.plot(
    x.sel(microphysics=mp).T,
    y,
    color=colors[mp],
    alpha=0.1,
)

add_second_x_axis(
    ax,
    new_xticks_func=lambda _x: (1e3 * 3600 * evap_rate_energy2evap_rate_mass(_x)).astype(int),
    xlabel=label_from_attrs(ds["evaporation_rate_energy"], return_units=False)
    + r" $\left[mg \, m^{-3} h^{-1}\right]$",
    position="bottom",
    offset_position=("axes", -0.15),
)

# plt.xscale('log')
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}_vs_{y.name}_{mp}-normalized")

### Compare microphysics

In [ ]:
fig, axss = plt.subplots(1, 1, figsize=np.array((16, 9)) * 0.5)

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

color_data = -ds["evaporation_rate_energy"]

# color_data = color_data / color_data.sel(gridbox = 0) * 100
y = ds["relative_humidity"]
x = ds["liquid_water_content"]


cmap = "inferno_r"


norm = mcolors.Normalize(
    vmin=0,
    vmax=200,
)

for i, mp in enumerate(plot_microphysics):

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    ax.plot(
        x.sel(microphysics=mp).T,
        y.sel(microphysics=mp).T,
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=30,
        alpha=0.8,
        # label = labels[mp],
    )
    # ax.set_yscale('log')
    ax.set_xscale("log")
    ax.set_title(labels[mp])

cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)

fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}-{y.name}-color-{color_data.name}")

/tmp/ipykernel_3249832/3003218016.py:59: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


In [ ]:
fig, axss = plt.subplots(1, 1, figsize=np.array((16, 9)) * 0.5)

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

color_data = -ds["evaporation_rate_energy"]

color_data = (color_data / color_data.mean("gridbox") - 1) * 100
color_data.attrs["units"] = "\\%"
color_data.attrs["long_name"] = "Evaporation rate relative to column mean"
color_data.name = "evaporation_rate_energy_relative"

y = ds["relative_humidity"]
x = ds["liquid_water_content"]

cmap = "RdBu_r"
cmap = sns.diverging_palette(h_neg=250, h_pos=30, s=100, l=50, center="light", as_cmap=True, n=3)
cmap = cmap.resampled(10)

norm = mcolors.Normalize(
    vmin=-100,
    vmax=100,
)

for i, mp in enumerate(plot_microphysics):

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    ax.plot(
        x.sel(microphysics=mp).T,
        y.sel(microphysics=mp).T,
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=20,
        alpha=0.8,
        # label = labels[mp],
    )
    # ax.set_yscale('log')
    ax.set_xscale("log")
    ax.set_title(labels[mp])


cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(
    mappable=sc,
    cax=cax,
    label=label_from_attrs(color_data),
    ticks=np.arange(-100, 101, 20),
)
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}-{y.name}-color-{color_data.name}")

/tmp/ipykernel_3249832/3423506374.py:69: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/tmp/ipykernel_3249832/3423506374.py:70: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


In [ ]:
fig, axss = plt.subplots(1, 1, figsize=np.array((16, 9)) * 0.5)

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

x = ds["liquid_water_content"]
y = ds["mass_radius_mean"]
color_data = -ds["evaporation_rate_energy"]

# color_data = (color_data / color_data.mean('gridbox') - 1) * 100
# color_data.attrs["units"] = "\\%"
# color_data.attrs["long_name"] = "Evaporation rate relative to column mean"
# color_data.name = "evaporation_rate_energy_relative"

cmap = strength_cmap
# cmap = sns.diverging_palette(h_neg= 250, h_pos= 30, s= 100, l= 50, center = 'light', as_cmap=True, n = 3)
# cmap = cmap.resampled(10)

norm = mcolors.Normalize(
    vmin=0,
    vmax=150,
)

for i, mp in enumerate(plot_microphysics):

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    ax.plot(
        x.sel(microphysics=mp).T,
        y.sel(microphysics=mp).T,
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=20,
        alpha=0.8,
        # label = labels[mp],
    )
    # ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_title(labels[mp])


cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(
    mappable=sc,
    cax=cax,
    label=label_from_attrs(color_data),
    # ticks = np.arange(-100, 101, 20),
)
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}-{y.name}-color-{color_data.name}")

/tmp/ipykernel_3249832/255990355.py:67: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


### Evaporation histograms

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["evaporation_fraction"]
for mp in ["condensation"]:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 100, 5),
        label=label,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 100, 5),
        histtype="step",
    )


ax.axvline(100, color="black", linestyle="--")

ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"histogram_{x.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = -ds["source_precipitation"]
for mp in ["condensation"]:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 1, 0.025),
        label=label,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 1, 0.025),
        histtype="step",
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


add_second_x_axis(
    ax,
    new_xticks_func=lambda x: evaporation_precipitation2evaporation_energy(x).astype(int),
    xlabel=label_from_attrs(ds["source_energy"]),
    position="bottom",
    offset_position=("axes", -0.15),
)


# add_second_x_axis(
#     ax,
#     new_xticks_func=lambda x : (3600 * 1e3 * evaporation_precipitation2evaporation_mass(x)).astype(int),
#     xlabel="Evaporation rate [kg/m^2/h]",
# position = ("axes", -0.30)
# )

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")
fig.tight_layout()

fig.tight_layout()
save_figure(fig, fig_dir / f"histogram_{x.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

plot_microphysics = ["condensation"]

for mp in plot_microphysics:

    ax.scatter(
        ds.sel(microphysics=mp)["inflow_precipitation"],
        -ds.sel(microphysics=mp)["source_precipitation"],
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    ds.sel(microphysics=plot_microphysics)["inflow_precipitation"],
    -ds.sel(microphysics=plot_microphysics)["source_precipitation"],
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")


ax.set_xscale("log")
ax.set_yscale("log")
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 2.5e1])
ax.set_ylim(lims.min(), lims.max())
ax.set_xlim(1e-4, lims.max())

p_x_values = np.geomspace(lims.min(), lims.max(), 100)


for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    x = 15
    y = p * x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )
    y = 2e-4
    x = y / p
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(x, y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )

ax.set_xlabel(label_from_attrs(ds["inflow_precipitation"]))
ax.set_ylabel(label_from_attrs(ds["source_precipitation"]))

Text(0, 0.5, 'Evaporation $\\left[  mm \\, h^{-1}  \\right]$')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = -ds["source_precipitation"]
for mp in compare_microphysics:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 1.0, 0.025),
        label=label,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 1.0, 0.025),
        histtype="step",
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


add_second_x_axis(
    ax,
    new_xticks_func=lambda x: evaporation_precipitation2evaporation_energy(x).astype(int),
    xlabel=label_from_attrs(ds["source_energy"]),
    position="bottom",
    offset_position=("axes", -0.15),
)


# add_second_x_axis(
#     ax,
#     new_xticks_func=lambda x : (3600 * 1e3 * evaporation_precipitation2evaporation_mass(x)).astype(int),
#     xlabel="Evaporation rate [kg/m^2/h]",
# position = ("axes", -0.30)
# )

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")
fig.tight_layout()

fig.tight_layout()
save_figure(fig, fig_dir / f"all-histogram_{x.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["evaporation_fraction"]
for mp in compare_microphysics:

    data = x.sel(microphysics=mp)
    m, sem = mean_and_stderror_of_mean(
        data=data,
        dims="cloud_id",
    )

    label = f'{labels[mp]}: {m.values:.2f} ± {sem.values:.2f} ${data.attrs["units"]}$'

    ax.axvline(
        m,
        color=colors[mp],
        linestyle="-",
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=0.1,
        bins=np.arange(0, 100, 5),
        label=label,
    )

    ax.hist(
        data,
        color=colors[mp],
        alpha=1,
        linewidth=1,
        bins=np.arange(0, 100, 5),
        histtype="step",
    )


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(ds["evaporation_fraction"]))
ax.set_ylabel("Counts")

fig.tight_layout()
save_figure(fig, fig_dir / f"all_histogram_{x.name}")

### Correlation of properties

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(10, 3))


for i, (x, y) in enumerate(
    (
        (-ds["source_energy"], ds["relative_humidity"].mean("gridbox", keep_attrs=True)),
        (-ds["source_energy"], ds["cloud_liquid_water_content"]),
        (-ds["source_energy"], ds["cloud_mass_radius_mean"]),
    )
):

    ax: plt.Axes = axs[i]
    da_corr = xr.corr(x, y, dim="cloud_id")
    print(x.name, y.name, da_corr.shape)

    for mp in ["condensation"]:
        data = da_corr.sel(microphysics=mp)

        ax.scatter(
            x.sel(microphysics=mp).T,
            y.sel(microphysics=mp).T,
            linewidth=0.5,
            alpha=0.5,
            color=colors[mp],
            # marker = markers[mp],
            label=f"{labels[mp]}: R = {data:.2f}",
        )

    ax.set_xlabel(label_from_attrs(x))
    ax.set_ylabel(label_from_attrs(y))
    ax.legend()

fig.tight_layout()
save_figure(fig, fig_dir / f"correlation_scatter-{x.name}")

source_energy relative_humidity (5,)
source_energy cloud_liquid_water_content (5,)
source_energy cloud_mass_radius_mean (5,)


In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(10, 3))


for i, (x, y) in enumerate(
    (
        (ds["evaporation_fraction"], ds["relative_humidity"].mean("gridbox", keep_attrs=True)),
        (ds["evaporation_fraction"], ds["cloud_liquid_water_content"]),
        (ds["evaporation_fraction"], ds["cloud_mass_radius_mean"]),
    )
):

    ax: plt.Axes = axs[i]
    da_corr = xr.corr(x, y, dim="cloud_id")
    print(x.name, y.name, da_corr.shape)

    for mp in ["condensation"]:
        data = da_corr.sel(microphysics=mp)

        ax.scatter(
            x.sel(microphysics=mp).T,
            y.sel(microphysics=mp).T,
            linewidth=0.5,
            alpha=0.5,
            color=colors[mp],
            # marker = markers[mp],
            label=f"{labels[mp]}: R = {data:.2f}",
        )

    ax.set_xlabel(label_from_attrs(x))
    ax.set_ylabel(label_from_attrs(y))
    ax.legend()

fig.tight_layout()
save_figure(fig, fig_dir / f"correlation_scatter-{x.name}")

evaporation_fraction relative_humidity (5,)
evaporation_fraction cloud_liquid_water_content (5,)
evaporation_fraction cloud_mass_radius_mean (5,)


### Correlation of evaporation rate against the rest

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

dss = ds.sel(microphysics="condensation")

bins = np.arange(-1, 1.01, 0.05)

x = xr.corr(dss["liquid_water_content"], -dss["evaporation_rate_energy"], dim="gridbox")
x.attrs["units"] = r"\,"
x.attrs["long_name"] = "RWC Corr"
x.name = "lwc_correlation_coefficient"
m, sem = mean_and_stderror_of_mean(
    data=x,
    dims="cloud_id",
)

label = (
    label_from_attrs(x, return_units=False) + f': {m.values:.2f} ± {sem.values:.2f} ${x.attrs["units"]}$'
)

ax.axvline(
    m,
    color=colors[mp],
    linestyle="-",
)

ax.hist(
    x,
    color=colors[mp],
    alpha=0.1,
    bins=bins,
    label=label,
)

ax.hist(
    x,
    color=colors[mp],
    alpha=1,
    linewidth=1,
    bins=bins,
    histtype="step",
)

x = xr.corr(dss["relative_humidity"], -dss["evaporation_rate_energy"], dim="gridbox")
x.attrs["units"] = r"\,"
x.attrs["long_name"] = "RH Corr"
x.name = "rh_correlation_coefficient"
m, sem = mean_and_stderror_of_mean(
    data=x,
    dims="cloud_id",
)

label = (
    label_from_attrs(x, return_units=False) + f': {m.values:.2f} ± {sem.values:.2f} ${x.attrs["units"]}$'
)

ax.axvline(
    m,
    color="orange",
    linestyle="-",
)

ax.hist(
    x,
    color="orange",
    alpha=0.1,
    bins=bins,
    label=label,
)

ax.hist(
    x,
    color="orange",
    alpha=1,
    linewidth=1,
    bins=bins,
    histtype="step",
)

ax.axvline(
    xr.corr(ds["cloud_liquid_water_content"], -ds["source_energy"]),
    color="k",
    linestyle="-",
    label="Column int. correlation",
)


ax.legend(loc="upper right", handler_map=handler_map_alpha())
# ax.set_yscale('log')
# ax.set_xscale('log')


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel("Counts")

Text(0, 0.5, 'Counts')

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_liquid_water_content"]
y = -ds["source_energy"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
# ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}_vs_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

a = (ds["relative_humidity"] / ds["relative_humidity"].mean("gridbox")) - 1

x = (a.max("gridbox") - a.min("gridbox")) / ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"]) * 1e3
x.name = "relative_humidity_gradient"
x.attrs["units"] = r"\% km$^{-1}$"
x.attrs["long_name"] = "Relative humidity gradient"
a = (ds["evaporation_rate_energy"] / ds["evaporation_rate_energy"].mean("gridbox")) - 1

y = (a.max("gridbox") - a.min("gridbox")) / ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"]) * 1e3
y.name = "source_energy_gradient"
y.attrs["units"] = r"mW m$^{-3}$ km$^{-1}$"
y.attrs["long_name"] = "Evaporation rate gradient"

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
# ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}_vs_{y.name}")

## Impact of mean mass radius

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}_vs_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

test = np.geomspace(6e1, 2e3, 100)
ax.plot(test, 3.7e7 * test ** (-5 / 2), color="k", linestyle="--", label=r"$r^{-5/2}$")

ax.legend(loc="upper right")
ax.set_yscale("log")
ax.set_xscale("log")
ax.set_ylim(None, 100)
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"fit-log-{x.name}_vs_{y.name}")

## Combined influece RWC and MMR

In [ ]:
fig, axss = plt.subplots(1, 1, figsize=(7, 5))

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

color_data = -ds["source_energy"]
# color_data = color_data / color_data.sel(gridbox = 0) * 100
y = ds["cloud_mass_radius_mean"]
x = ds["cloud_liquid_water_content"]

color_data = color_data.sel(microphysics=plot_microphysics)
x = x.sel(microphysics=plot_microphysics)
y = y.sel(microphysics=plot_microphysics)


cmap = strength_cmap


norm = mcolors.Normalize(
    vmin=color_data.compute().quantile(0.1).values, vmax=color_data.compute().quantile(0.9).values
)

for i, mp in enumerate(plot_microphysics):

    # cmap = sns.cubehelix_palette(start=i * 0.2, rot=-0.5, as_cmap=True)

    c = color_data.sel(microphysics=mp)
    ax = axs_flat[i]

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=70,
        # label = labels[mp],
    )
    # ax.set_yscale('log')
    ax.set_xscale("log")
    ax.set_title(labels[mp])


cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}-{y.name}-color-{color_data.name}")

/tmp/ipykernel_3249832/766767560.py:52: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


In [ ]:
fig, axss = plt.subplots(1, 1, figsize=(7, 5))

axss = np.array(axss)
axs_flat = axss.flatten()

plot_microphysics = ["condensation"]

color_data = ds["evaporation_fraction"]
# color_data = color_data / color_data.sel(gridbox = 0) * 100
y = ds["cloud_mass_radius_mean"]
x = ds["cloud_liquid_water_content"]

color_data = color_data.sel(microphysics=plot_microphysics)
x = x.sel(microphysics=plot_microphysics)
y = y.sel(microphysics=plot_microphysics)

cmap = strength_cmap


norm = mcolors.Normalize(
    vmin=color_data.compute().quantile(0.1).values, vmax=color_data.compute().quantile(0.9).values
)

for i, mp in enumerate(plot_microphysics):

    # cmap = sns.cubehelix_palette(start=i * 0.2, rot=-0.5, as_cmap=True)

    c = color_data.sel(microphysics=mp)
    ax = axs_flat[i]

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=70,
        # label = labels[mp],
    )
    # ax.set_yscale('log')
    ax.set_xscale("log")
    ax.set_title(labels[mp])


cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}-{y.name}-color-{color_data.name}")

/tmp/ipykernel_3249832/866614276.py:51: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


### Reconstruct Evaporation from LWC and MMR

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

plot_microphysics = ["condensation"]

color_data = ds["cloud_liquid_water_content"]
# color_data = color_data / color_data.sel(gridbox = 0) * 100
x = -ds["source_precipitation"]

y = ds["cloud_mass_radius_mean"] ** (-5 / 2) * ds["cloud_liquid_water_content"]
y.attrs.update(long_name="Reconstruction", units="m^{-5/2} * kg / m^3")
y.name = "reconstructed_evaporation-using-radius"

cmap = strength_cmap


norm = mcolors.Normalize(
    vmin=color_data.compute().quantile(0.1).values, vmax=color_data.compute().quantile(0.9).values
)

for i, mp in enumerate(plot_microphysics):

    print(mp)

    R = float(xr.corr(x.sel(microphysics=mp), y.sel(microphysics=mp), dim="cloud_id"))

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=colors[mp],
        marker=markers[mp],
        s=70,
        label=f"{labels[mp]}: R = {R:.2f}",
    )
    # ax.set_yscale("log")
    # ax.set_xscale("log")
    ax.set_title(labels[mp])

ax.legend(loc="upper right")

# cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
# fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))

fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# ax.set_xlim(1e-4, None)
fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}-{y.name}-color-{color_data.name}")

condensation


# Microphysics


## Comparison of microphysics profiles

### Physical gridbox space

In [ ]:
y_ticks = np.arange(0, 1300, 250)

fig = plt.figure()
gs = fig.add_gridspec(nrows=2, ncols=4)

ax = fig.add_subplot(gs[:, :2])
ax0 = fig.add_subplot(gs[0, 2])
ax1 = fig.add_subplot(gs[0, 3], sharey=ax0, sharex=ax0)
ax2 = fig.add_subplot(gs[1, 2], sharey=ax0, sharex=ax0)
ax3 = fig.add_subplot(gs[1, 3], sharey=ax0, sharex=ax0)

axs = dict(
    zip(
        compare_microphysics,
        [ax0, ax1, ax2, ax3],
    )
)

for mp in compare_microphysics:

    md = ds.sel(microphysics=mp)["evaporation_rate_energy"]
    md_mean, md_std = mean_and_stderror_of_mean(
        data=md,
        dims="cloud_id",
    )

    z = ds.sel(microphysics=mp)["gridbox_coord3"]
    z_mean, z_std = mean_and_stderror_of_mean(
        data=z,
        dims="cloud_id",
    )

    axs[mp].plot(
        md.T,
        z.T,
        color=light_colors[mp],
        alpha=0.3,
    )
    axs[mp].plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    ax.plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )
    ax.fill_betweenx(
        z_mean,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.1,
        color=colors[mp],
    )
ax.set_yticks(y_ticks, y_ticks)
ax.legend()
ax.set_xlim(-150, 0)
ax0.set_xlim(-150, 0)

fig.supxlabel(label_from_attrs(ds["evaporation_rate_energy"]))
fig.supylabel("Height [m]")
fig.suptitle("Evaporation profiles")
fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / "evaporation_profiles")

In [ ]:
y_ticks = np.arange(0, 1300, 250)

fig = plt.figure()
gs = fig.add_gridspec(nrows=2, ncols=4)

ax = fig.add_subplot(gs[:, :2])
ax0 = fig.add_subplot(gs[0, 2])
ax1 = fig.add_subplot(gs[0, 3], sharey=ax0, sharex=ax0)
ax2 = fig.add_subplot(gs[1, 2], sharey=ax0, sharex=ax0)
ax3 = fig.add_subplot(gs[1, 3], sharey=ax0, sharex=ax0)

axs = dict(
    zip(
        compare_microphysics,
        [ax0, ax1, ax2, ax3],
    )
)

for mp in compare_microphysics:

    md = ds_diff.sel(microphysics=mp)["evaporation_rate_energy"]
    md_mean, md_std = mean_and_stderror_of_mean(
        data=md,
        dims="cloud_id",
    )

    z = ds.sel(microphysics=mp)["gridbox_coord3"]
    z_mean, z_std = mean_and_stderror_of_mean(
        data=z,
        dims="cloud_id",
    )

    axs[mp].plot(
        md.T,
        z.T,
        color=light_colors[mp],
        alpha=0.3,
    )
    axs[mp].plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])
    # axs[mp].set_xscale('symlog', linthresh = 1e-8)
    ax.plot(
        md_mean,
        z_mean,
        label=labels[mp],
        color=dark_colors[mp],
    )
    ax.fill_betweenx(
        z_mean,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.3,
        color=colors[mp],
    )


ax.set_yticks(y_ticks, y_ticks)

ax.legend(loc="upper right")
ax.set_xlim(-15, 15)
ax0.set_xlim(-30, 30)


fig.supxlabel(label_from_attrs(ds["evaporation_rate_energy"]))
fig.supylabel("Height [m]")
fig.suptitle("Evpoartion profiles | Difference to Evaporation only setup ")
fig.tight_layout()

fig.tight_layout()
save_figure(fig, fig_dir / "evaporation_profiles_diff")

In [ ]:
plot_microphysics = [
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
]

N = len(plot_microphysics) + 1
width_ratios = np.ones(N)
width_ratios[-1] = 0.05

fig, axs = plt.subplots(nrows=1, ncols=N, figsize=np.array((12 * 3, 9)) * 0.5, width_ratios=width_ratios)

cax = axs[-1]
axs_flat = axs[:-1].flatten()

x = ds["liquid_water_content"]
y = ds["relative_humidity"]
c = ds["evaporation_rate_energy"]
attrs = c.attrs.copy()
c_refernce = c.sel(microphysics="condensation")
c = (c / c_refernce - 1) * 100
c.attrs.update(
    long_name=f"{attrs['long_name']}\nrelative difference to {labels['condensation']}",
    units=r"\%",
)
color_data = c

cmap = strength_cmap
cmap = sns.diverging_palette(h_neg=250, h_pos=30, s=100, l=35, center="light", as_cmap=True)
# cmap = cmap.resampled(10)

norm = mcolors.SymLogNorm(
    linthresh=10,
    linscale=0.5,
    vmin=-100,
    vmax=100,
)

for i, mp in enumerate(plot_microphysics):
    print(mp)

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    ax.plot(
        x.sel(microphysics=mp).T,
        y.sel(microphysics=mp).T,
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=20,
        alpha=0.8,
        # label = labels[mp],
    )
    # ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_title(labels[mp])


fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
fig.colorbar(
    mappable=sc,
    cax=cax,
    label=label_from_attrs(color_data),
    # ticks = np.arange(-100, 101, 20),
)
fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"all-diff-{x.name}-{y.name}-color-{color_data.name}")

collision_condensation
coalbure_condensation_small
coalbure_condensation_large


### Normalized

In [ ]:
y_ticks = np.arange(0, 1.01, 0.25)

fig = plt.figure()
gs = fig.add_gridspec(nrows=2, ncols=4)

ax = fig.add_subplot(gs[:, :2])
ax0 = fig.add_subplot(gs[0, 2])
ax1 = fig.add_subplot(gs[0, 3], sharey=ax0, sharex=ax0)
ax2 = fig.add_subplot(gs[1, 2], sharey=ax0, sharex=ax0)
ax3 = fig.add_subplot(gs[1, 3], sharey=ax0, sharex=ax0)

axs = dict(
    zip(
        compare_microphysics,
        [ax0, ax1, ax2, ax3],
    )
)

x = ds_interpolated["evaporation_rate_energy"]
y = ds_interpolated["normalized_gridbox_coord3"]

for mp in compare_microphysics:

    md = x.sel(microphysics=mp)
    md_mean, md_std = mean_and_stderror_of_mean(
        data=md,
        dims="cloud_id",
    )

    axs[mp].plot(
        md.T,
        y.T,
        color=light_colors[mp],
        alpha=0.3,
    )
    axs[mp].plot(
        md_mean,
        y,
        label=labels[mp],
        color=dark_colors[mp],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    ax.plot(
        md_mean,
        y,
        label=labels[mp],
        color=dark_colors[mp],
    )
    ax.fill_betweenx(
        y,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.1,
        color=colors[mp],
    )
ax.set_yticks(y_ticks, y_ticks)
ax.legend()
ax.set_xlim(-200, 0)
ax0.set_xlim(-400, 0)

fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
fig.suptitle("Evaporation profiles")
fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / "evaporation_profiles-normalized")

In [ ]:
y_ticks = np.arange(0, 1.01, 0.25)

fig = plt.figure()
gs = fig.add_gridspec(nrows=2, ncols=4)

ax = fig.add_subplot(gs[:, :2])
ax0 = fig.add_subplot(gs[0, 2])
ax1 = fig.add_subplot(gs[0, 3], sharey=ax0, sharex=ax0)
ax2 = fig.add_subplot(gs[1, 2], sharey=ax0, sharex=ax0)
ax3 = fig.add_subplot(gs[1, 3], sharey=ax0, sharex=ax0)

axs = dict(
    zip(
        compare_microphysics,
        [ax0, ax1, ax2, ax3],
    )
)

x = ds_interpolated["evaporation_rate_energy"]
attrs = x.attrs.copy()
x = x - x.sel(microphysics="condensation")
x.attrs.update(attrs)
y = ds_interpolated["normalized_gridbox_coord3"]

for mp in compare_microphysics:

    md = x.sel(microphysics=mp)
    md_mean, md_std = mean_and_stderror_of_mean(
        data=md,
        dims="cloud_id",
    )

    axs[mp].plot(
        md.T,
        y.T,
        color=light_colors[mp],
        alpha=0.3,
    )
    axs[mp].plot(
        md_mean,
        y,
        label=labels[mp],
        color=dark_colors[mp],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    ax.plot(
        md_mean,
        y,
        label=labels[mp],
        color=dark_colors[mp],
    )
    ax.fill_betweenx(
        y,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.1,
        color=colors[mp],
    )
ax.set_yticks(y_ticks, y_ticks)
ax.legend()
ax.set_xlim(-20, 20)
ax0.set_xlim(-100, 100)

fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
fig.suptitle("Evpoartion profiles | Difference to Evaporation only setup ")
fig.tight_layout()

fig.tight_layout()
save_figure(fig, fig_dir / "evaporation_profiles_diff-normalized")

In [ ]:
y_ticks = np.arange(0, 1.01, 0.25)

fig = plt.figure()
gs = fig.add_gridspec(nrows=2, ncols=4)

ax = fig.add_subplot(gs[:, :2])
ax0 = fig.add_subplot(gs[0, 2])
ax1 = fig.add_subplot(gs[0, 3], sharey=ax0, sharex=ax0)
ax2 = fig.add_subplot(gs[1, 2], sharey=ax0, sharex=ax0)
ax3 = fig.add_subplot(gs[1, 3], sharey=ax0, sharex=ax0)

axs = dict(
    zip(
        compare_microphysics,
        [ax0, ax1, ax2, ax3],
    )
)

x = ds_interpolated["evaporation_rate_energy"]
x_refernce = x.sel(microphysics="condensation")
attrs = x.attrs.copy()

x = (x / x_refernce - 1) * 100
x.attrs.update(
    long_name=f"{attrs['long_name']}\nrelative difference to {labels['condensation']}",
    units=r"\%",
)
y = ds_interpolated["normalized_gridbox_coord3"]

for mp in compare_microphysics:

    md = x.sel(microphysics=mp)
    md_mean, md_std = mean_and_stderror_of_mean(
        data=md,
        dims="cloud_id",
    )

    axs[mp].plot(
        md.T,
        y.T,
        color=light_colors[mp],
        alpha=0.3,
    )
    axs[mp].plot(
        md_mean,
        y,
        label=labels[mp],
        color=dark_colors[mp],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    ax.plot(
        md_mean,
        y,
        label=labels[mp],
        color=dark_colors[mp],
    )
    ax.fill_betweenx(
        y,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.1,
        color=colors[mp],
    )
ax.set_yticks(y_ticks, y_ticks)
ax.legend(loc="upper left")
ax.set_xlim(-10, 10)

ax0.set_xlim(-50, 100)

for _ax in axs.values():
    _ax.axvline(0, color="k", linestyle="--", alpha=0.5)
ax0.axvline(0, color="k", linestyle="--", alpha=0.5)

fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
fig.suptitle("Evpoartion profiles | Difference to Evaporation only setup ")
fig.tight_layout()

fig.tight_layout()
save_figure(fig, fig_dir / "evaporation_profiles_diff-percentile-normalized")

In [ ]:
plot_microphysics = [
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
]

N = len(plot_microphysics) + 1
width_ratios = np.ones(N)
width_ratios[-1] = 0.05

fig, axs = plt.subplots(nrows=1, ncols=N, figsize=np.array((12 * 3, 9)) * 0.5, width_ratios=width_ratios)

cax = axs[-1]
axs_flat = axs[:-1].flatten()

x = ds_interpolated["liquid_water_content"]
y = ds_interpolated["relative_humidity"]
c = ds_interpolated["evaporation_rate_energy"]
attrs = c.attrs.copy()
c_refernce = c.sel(microphysics="condensation")
c = (c / c_refernce - 1) * 100
c.attrs.update(
    long_name=f"{attrs['long_name']}\nrelative difference to {labels['condensation']}",
    units=r"\%",
)
color_data = c

cmap = strength_cmap
cmap = sns.diverging_palette(h_neg=250, h_pos=30, s=100, l=35, center="light", as_cmap=True)
# cmap = cmap.resampled(10)

norm = mcolors.SymLogNorm(
    linthresh=10,
    linscale=0.5,
    vmin=-100,
    vmax=100,
)
# norm = None

for i, mp in enumerate(plot_microphysics):
    print(mp)

    c = color_data.sel(microphysics=mp)

    ax = axs_flat[i]

    ax.plot(
        x.sel(microphysics=mp).T,
        y.sel(microphysics=mp).T,
        color="grey",
        linewidth=0.2,
        alpha=0.5,
        zorder=0,
    )

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=c,
        cmap=cmap,
        norm=norm,
        marker=".",
        s=20,
        alpha=0.8,
        # label = labels[mp],
    )
    # ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_title(labels[mp])


fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
fig.colorbar(
    mappable=sc,
    cax=cax,
    label=label_from_attrs(color_data),
    # ticks = np.arange(-100, 101, 20),
)
fig.tight_layout()
fig.tight_layout()
save_figure(fig, fig_dir / f"all-diff-{x.name}-{y.name}-color-{color_data.name}-normalized")

collision_condensation
coalbure_condensation_small
coalbure_condensation_large


## Microphysics differences explained

### Mean mass radius

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

for mp in compare_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=selected_microphysics),
    y.sel(microphysics=selected_microphysics),
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


test = np.geomspace(6e1, 2e3, 100)
ax.plot(test, 3.7e7 * test ** (-5 / 2), color="k", linestyle="--", label=r"$r^{-5/2}$")

ax.legend(loc="upper right")
ax.set_yscale("log")
ax.set_xscale("log")
ax.set_ylim(None, 100)
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"all-fit-log_{x.name}_vs_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

for mp in compare_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=selected_microphysics),
    y.sel(microphysics=selected_microphysics),
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


test = np.geomspace(6e1, 2e3, 100)
ax.plot(test, 3.7e7 * test ** (-5 / 2), color="k", linestyle="--", label=r"$r^{-5/2}$")

ax.legend(loc="upper right")
# ax.set_yscale("log")
# ax.set_xscale("log")
ax.set_ylim(0, 100)
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"all-fit-linear_{x.name}_vs_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = -ds["source_precipitation"]

for mp in compare_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=selected_microphysics),
    y.sel(microphysics=selected_microphysics),
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper right")
ax.set_yscale("log")
ax.set_xscale("log")
# ax.set_ylim(0, 100)
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"all-fit-linear_{x.name}_vs_{y.name}")

### Cloud rain water content

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_liquid_water_content"]
y = ds["evaporation_fraction"]

for mp in ds["microphysics"].values:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=list(compare_microphysics)),
    y.sel(microphysics=list(compare_microphysics)),
    color="grey",
    linewidth=0.2,
    alpha=0.2,
    zorder=0,
)

ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"all_{x.name}_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_liquid_water_content"]
y = -ds["source_precipitation"]

for mp in compare_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=selected_microphysics),
    y.sel(microphysics=selected_microphysics),
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

ax.set_yscale("linear")
fig.tight_layout()
save_figure(fig, fig_dir / f"all_{x.name}_{y.name}-linear")

ax.set_yscale("log")
fig.tight_layout()
save_figure(fig, fig_dir / f"all_{x.name}_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_liquid_water_content"]
y = -ds["source_precipitation"]

for mp in compare_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=selected_microphysics),
    y.sel(microphysics=selected_microphysics),
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")
# ax.set_yscale('log')
ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

ax.set_ylim(None, 1.4)

fig.tight_layout()
save_figure(fig, fig_dir / f"all_{x.name}_{y.name}-linear-zoom")

### Relative differences

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_liquid_water_content"]
y = -ds["source_precipitation"]
attrs = y.attrs.copy()
y = (y / y.sel(microphysics="condensation") - 1) * 100
y.name = "source_precipitation_relative_difference"
y.attrs.update(
    long_name=f"{attrs['long_name']}\nrelative difference to {labels['condensation']}", units=r"\%"
)


for mp in compare_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=list(compare_microphysics)),
    y.sel(microphysics=list(compare_microphysics)),
    color="grey",
    linewidth=0.2,
    alpha=0.2,
    zorder=0,
)

ax.legend(loc="upper left")

ticks = 10.0 ** np.arange(0, 4, 1)
ticks = np.concatenate([np.flip(-ticks), [0], ticks])
ticks = ticks.astype(int)

ax.set_yscale("symlog", linthresh=1, linscale=0.5)
ax.set_xscale("log")
ax.set_yticks(ticks, ticks)


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"all_relative-difference-{x.name}_{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_mass_radius_mean"]
y = -ds["source_precipitation"]
attrs = y.attrs.copy()
y = (y / y.sel(microphysics="condensation") - 1) * 100
y.name = "source_precipitation_relative_difference"
y.attrs.update(
    long_name=f"{attrs['long_name']}\nrelative difference to {labels['condensation']}", units=r"\%"
)


for mp in compare_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=list(compare_microphysics)),
    y.sel(microphysics=list(compare_microphysics)),
    color="grey",
    linewidth=0.2,
    alpha=0.2,
    zorder=0,
)

ax.legend(loc="upper left")

ticks = 10.0 ** np.arange(0, 4, 1)
ticks = np.concatenate([np.flip(-ticks), [0], ticks])
ticks = ticks.astype(int)

ax.set_yscale("symlog", linthresh=1, linscale=0.5)
ax.set_xscale("log")
ax.set_yticks(ticks, ticks)


ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"all_relative-difference-{x.name}_{y.name}")

### Combined plot of evaporation rate and evaporation fraction

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["inflow_precipitation"]
y = -ds["source_precipitation"]

selected_microphysics = ["condensation"]

for mp in selected_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=selected_microphysics),
    y.sel(microphysics=selected_microphysics),
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")


ax.set_xscale("log")
ax.set_yscale("log")
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 2.5e1])
ax.set_ylim(lims.min(), lims.max())
ax.set_xlim(1e-4, lims.max())

p_x_values = np.geomspace(lims.min(), lims.max(), 100)


for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    _x = 15
    _y = p * _x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(_x, _y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )
    _y = 2e-4
    _x = _y / p
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(_x, _y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"combined-{x.name}-{y.name}")

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["inflow_precipitation"]
y = -ds["source_precipitation"]

selected_microphysics = list(compare_microphysics)

for mp in selected_microphysics:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.plot(
    x.sel(microphysics=selected_microphysics),
    y.sel(microphysics=selected_microphysics),
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="upper left")


ax.set_xscale("log")
ax.set_yscale("log")
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 2.5e1])
ax.set_ylim(lims.min(), lims.max())
ax.set_xlim(1e-4, lims.max())

p_x_values = np.geomspace(lims.min(), lims.max(), 100)


for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    _x = 15
    _y = p * _x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(_x, _y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )
    _y = 2e-4
    _x = _y / p
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(_x, _y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=10,
        **style,
    )

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"combined-all-{x.name}-{y.name}")

### Reconstruction

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

plot_microphysics = ["condensation"]

color_data = ds["cloud_liquid_water_content"]
# color_data = color_data / color_data.sel(gridbox = 0) * 100
x = -ds["source_precipitation"]

y = ds["cloud_mass_radius_mean"] ** (-5 / 2) * ds["cloud_liquid_water_content"]
y.attrs.update(long_name="Reconstruction", units="m^{-5/2} * kg / m^3")
y.name = "reconstructed_evaporation-using-radius"

for i, mp in enumerate(plot_microphysics):

    print(mp)

    R = float(xr.corr(x.sel(microphysics=mp), y.sel(microphysics=mp), dim="cloud_id"))

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=colors[mp],
        marker=markers[mp],
        s=70,
        label=f"{labels[mp]}: R = {R:.2f}",
    )
    # ax.set_yscale("log")
    # ax.set_xscale("log")
    # ax.set_title(labels[mp])

ax.legend(loc="center right")

# cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
# fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))

fig.suptitle(r"Reconstruction of evaporation rate using $ r^{-5/2} \cdot RWC$")
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
ax.set_xlim(0, 1.4)
fig.tight_layout()
save_figure(fig, fig_dir / f"reconstruction-{x.name}-{y.name}")

condensation


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

plot_microphysics = compare_microphysics

color_data = ds["cloud_liquid_water_content"]
# color_data = color_data / color_data.sel(gridbox = 0) * 100
x = -ds["source_precipitation"]

y = ds["cloud_mass_radius_mean"] ** (-5 / 2) * ds["cloud_liquid_water_content"]
y.attrs.update(long_name="Reconstruction", units="m^{-5/2} * kg / m^3")
y.name = "reconstructed_evaporation-using-radius"

for i, mp in enumerate(plot_microphysics):

    print(mp)

    R = float(xr.corr(x.sel(microphysics=mp), y.sel(microphysics=mp), dim="cloud_id"))

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=colors[mp],
        marker=markers[mp],
        s=70,
        label=f"{labels[mp]}: R = {R:.2f}",
    )
    # ax.set_yscale("log")
    # ax.set_xscale("log")
    # ax.set_title(labels[mp])

ax.plot(
    x.sel(microphysics=selected_microphysics),
    y.sel(microphysics=selected_microphysics),
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="center right")

# cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
# fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))

fig.suptitle(r"Reconstruction of evaporation rate using $ r^{-5/2} \cdot RWC$")
fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
ax.set_xlim(0, 1.4)
fig.tight_layout()
save_figure(fig, fig_dir / f"reconstruction-all-{x.name}-{y.name}")

condensation
collision_condensation
coalbure_condensation_small
coalbure_condensation_large


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

plot_microphysics = compare_microphysics

color_data = ds["cloud_liquid_water_content"]
# color_data = color_data / color_data.sel(gridbox = 0) * 100
x = -ds["source_precipitation"]

y = ds["cloud_mass_radius_mean"] ** (-5 / 2) * ds["cloud_liquid_water_content"]
y.attrs.update(long_name="Reconstruction", units="m^{-5/2} * kg / m^3")
y.name = "reconstructed_evaporation-using-radius"

for i, mp in enumerate(plot_microphysics):

    print(mp)

    R = float(xr.corr(x.sel(microphysics=mp), y.sel(microphysics=mp), dim="cloud_id"))

    sc = ax.scatter(
        x=x.sel(microphysics=mp),
        y=y.sel(microphysics=mp),
        c=colors[mp],
        marker=markers[mp],
        s=70,
        label=f"{labels[mp]}: R = {R:.2f}",
    )
    # ax.set_yscale("log")
    # ax.set_xscale("log")
    ax.set_title(labels[mp])

ax.plot(
    x.sel(microphysics=selected_microphysics),
    y.sel(microphysics=selected_microphysics),
    color="grey",
    linewidth=0.2,
    alpha=0.5,
    zorder=0,
)


ax.legend(loc="center right")

# cax = fig.add_axes([0.92, 0.1, 0.015, 0.8])
# fig.colorbar(mappable=sc, cax=cax, label=label_from_attrs(color_data))

fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
ax.set_xlim(0, 1.4)
fig.tight_layout()
save_figure(fig, fig_dir / f"all-{x.name}-{y.name}-color-{color_data.name}")

condensation
collision_condensation
coalbure_condensation_small
coalbure_condensation_large


# Appendix  

## Cloud DSD and MSD

### Normalized gridbox

In [ ]:
# number_heights = 3

# fig = plt.figure(layout='constrained', figsize=(12, 10))
# subfigs = fig.subfigures(nrows = 1, ncols = 2, wspace=0.07)

# axs_dsd = subfigs[0].subplots(nrows = number_heights + 1, ncols = 1, sharey=False, sharex=True)
# axs_msd = subfigs[1].subplots(nrows = number_heights + 1, ncols = 1, sharey=False, sharex=True)

# x = ds_interpolated["radius_bins"]
# x.attrs.update({"long_name": "Radius", "units": "µm"})

# surface = ds['gridbox_coord3_norm'].sel(
#                 gridbox = 0,
#                 )
# sub_cloud_top = ds['gridbox_coord3_norm'].sel(
#                 gridbox = ds['max_gridbox'] - 1
# )

# num_conc = ds_interpolated['xi_temporal_mean']
# num_conc.attrs.update({"long_name": "Number concentration", "units": "m^{-3}"})
# num_conc_diff = num_conc.sel(
#             normalized_gridbox_coord3 = sub_cloud_top, method = 'nearest'
#         ) - num_conc.sel(normalized_gridbox_coord3 = surface, method = 'nearest'
#         )
# mass_conc = ds_interpolated['mass_represented_temporal_mean']
# mass_conc.attrs.update({"long_name": "Mass concentration", "units": "kg m^{-3}"})
# mass_conc_diff = mass_conc.sel(
#             normalized_gridbox_coord3 = sub_cloud_top, method = 'nearest'
#         ) - mass_conc.sel(normalized_gridbox_coord3 = surface, method = 'nearest'
#         )

# for microphysic in compare_microphysics:

#     for da, axs in zip([num_conc, mass_conc], [axs_dsd, axs_msd]) :

#         for i, coord in enumerate(np.flip(np.linspace(0.1, 0.9, number_heights))) :
#             ax = axs[i]
#             _y = da.sel(
#                 normalized_gridbox_coord3 = coord, method = 'nearest'
#             ).sel(
#                 microphysics = microphysic
#             )

#             ma, q25, q75 = _y.quantile([0.5, 0.25, 0.75], dim="cloud_id")
#             ax.plot(
#                 x,
#                 ma,
#                 color= dark_colors[microphysic],
#                 alpha = 1,
#                 linestyle = '-',
#                 label = labels[microphysic],
#                 # marker = '.'
#             );
#             ax.fill_between(
#                 x,
#                 q25,
#                 q75,
#                 color= colors[microphysic],
#                 alpha = 0.2,
#                 linestyle = '-',
#                 # label = labels[microphysic],
#                 # marker = '.'
#             );
#             ax.set_title(f"Height {coord:.2f}")

#     for da, axs in zip([num_conc_diff, mass_conc_diff], [axs_dsd, axs_msd]) :
#         ax = axs[-1]
#         _y = da.sel(
#                 microphysics = microphysic
#             )

#         ma, q25, q75 = _y.quantile([0.5, 0.25, 0.75], dim="cloud_id")
#         ax.plot(
#             x,
#             ma,
#             color= dark_colors[microphysic],
#             alpha = 1,
#             linestyle = '-',
#             label = labels[microphysic],
#             # marker = '.'
#         );
#         ax.fill_between(
#             x,
#             q25,
#             q75,
#             color= colors[microphysic],
#             alpha = 0.2,
#             linestyle = '-',
#             # label = labels[microphysic],
#             # marker = '.'
#         );
#         ax.set_title(f"Height difference")

# for ax in axs_dsd:
#     ax.set_xlabel(label_from_attrs(x))
#     ax.set_ylabel(label_from_attrs(num_conc))
#     ax.set_xscale('log')
# for ax in axs_dsd[:-1]:
#     ax.set_ylim(0, 7e6)
# axs_dsd[-1].set_ylim(-2e6, 2e6)


# for ax in axs_msd:
#     ax.set_xlabel(label_from_attrs(x))
#     ax.set_ylabel(label_from_attrs(mass_conc))
#     ax.set_xscale('log')
# for ax in axs_msd[:-1]:
#     ax.set_ylim(0, 1)
# axs_msd[-1].set_ylim(-0.1, 0.1)

# for ax in np.concatenate([axs_dsd, axs_msd]):
#     ax.set_xlim(20, None)
#     ax.legend(loc= 'upper left')

#     # axs[0, 0].set_yscale('log')

# fig.suptitle("All microphysics")

In [ ]:
# number_heights = 3

# fig = plt.figure(layout='constrained', figsize=(12, 10))
# subfigs = fig.subfigures(nrows = 1, ncols = 2, wspace=0.07)

# axs_dsd = subfigs[0].subplots(nrows = number_heights + 1, ncols = 1, sharey=False, sharex=True)
# axs_msd = subfigs[1].subplots(nrows = number_heights + 1, ncols = 1, sharey=False, sharex=True)

# x = ds_interpolated["radius_bins"]
# x.attrs.update({"long_name": "Radius", "units": "µm"})

# surface = ds['gridbox_coord3_norm'].sel(
#                 gridbox = 0,
#                 )
# sub_cloud_top = ds['gridbox_coord3_norm'].sel(
#                 gridbox = ds['max_gridbox'] - 1
# )

# num_conc = ds_interpolated['xi_temporal_mean']
# num_conc = (num_conc / num_conc.sel(microphysics = 'condensation') - 1) * 100
# num_conc.attrs.update({"long_name": "Number concentration", "units": "m^{-3}"})
# num_conc_diff = num_conc.sel(
#             normalized_gridbox_coord3 = sub_cloud_top, method = 'nearest'
#         ) - num_conc.sel(normalized_gridbox_coord3 = surface, method = 'nearest'
#         )

# mass_conc = ds_interpolated['mass_represented_temporal_mean']
# mass_conc = (mass_conc / mass_conc.sel(microphysics = 'condensation') - 1) * 100

# mass_conc.attrs.update({"long_name": "Mass concentration", "units": "kg m^{-3}"})
# mass_conc_diff = mass_conc.sel(
#             normalized_gridbox_coord3 = sub_cloud_top, method = 'nearest'
#         ) - mass_conc.sel(normalized_gridbox_coord3 = surface, method = 'nearest'
#         )

# for microphysic in compare_microphysics:

#     for da, axs in zip([num_conc, mass_conc], [axs_dsd, axs_msd]) :

#         for i, coord in enumerate(np.flip(np.linspace(0.1, 0.9, number_heights))) :
#             ax = axs[i]
#             _y = da.sel(
#                 normalized_gridbox_coord3 = coord, method = 'nearest'
#             ).sel(
#                 microphysics = microphysic
#             )

#             ma, q25, q75 = _y.quantile([0.5, 0.25, 0.75], dim="cloud_id")
#             ax.plot(
#                 x,
#                 ma,
#                 color= dark_colors[microphysic],
#                 alpha = 1,
#                 linestyle = '-',
#                 label = labels[microphysic],
#                 # marker = '.'
#             );
#             ax.fill_between(
#                 x,
#                 q25,
#                 q75,
#                 color= colors[microphysic],
#                 alpha = 0.2,
#                 linestyle = '-',
#                 # label = labels[microphysic],
#                 # marker = '.'
#             );
#             ax.set_title(f"Height {coord:.2f}")

#     for da, axs in zip([num_conc_diff, mass_conc_diff], [axs_dsd, axs_msd]) :
#         ax = axs[-1]
#         _y = da.sel(
#                 microphysics = microphysic
#             )

#         ma, q25, q75 = _y.quantile([0.5, 0.25, 0.75], dim="cloud_id")
#         ax.plot(
#             x,
#             ma,
#             color= dark_colors[microphysic],
#             alpha = 1,
#             linestyle = '-',
#             label = labels[microphysic],
#             # marker = '.'
#         );
#         ax.fill_between(
#             x,
#             q25,
#             q75,
#             color= colors[microphysic],
#             alpha = 0.2,
#             linestyle = '-',
#             # label = labels[microphysic],
#             # marker = '.'
#         );
#         ax.set_title(f"Height difference")

# for ax in axs_dsd:
#     ax.set_xlabel(label_from_attrs(x))
#     ax.set_ylabel(label_from_attrs(num_conc))
#     ax.set_xscale('log')
# for ax in axs_dsd[:-1]:
#     ax.set_ylim(-100, 100)
# axs_dsd[-1].set_ylim(-100, 100)


# for ax in axs_msd:
#     ax.set_xlabel(label_from_attrs(x))
#     ax.set_ylabel(label_from_attrs(mass_conc))
#     ax.set_xscale('log')
# for ax in axs_msd[:-1]:
#     ax.set_ylim(-100, 100)
# axs_msd[-1].set_ylim(-100, 100)

# for ax in np.concatenate([axs_dsd, axs_msd]):
#     ax.set_xlim(20, None)
#     ax.legend(loc= 'upper left')

#     # axs[0, 0].set_yscale('log')

# fig.suptitle("All microphysics")

### Physical space

In [ ]:
# fig = plt.figure(layout='constrained', figsize=(12, 10))
# subfigs = fig.subfigures(nrows = 1, ncols = 2, wspace=0.07)

# axs_dsd = subfigs[0].subplots(nrows = 4, ncols = 1, sharey=False, sharex=True)
# axs_msd = subfigs[1].subplots(nrows = 4, ncols = 1, sharey=False, sharex=True)

# for microphysic in compare_microphysics:

#     x = ds["radius_bins"]
#     x.attrs.update({"long_name": "Radius", "units": "µm"})
#     number_concentration = ds['xi_temporal_mean'].sel(microphysics=microphysic).T
#     number_concentration.attrs.update({"long_name": "Number concentration", "units": "m^{-3}"})

#     number_concentration_top = number_concentration.sel(gridbox=ds["max_gridbox"].sel(microphysics=microphysic) - 1)
#     number_concentration_ten = number_concentration.sel(gridbox=10)
#     number_concentration_surface = number_concentration.sel(gridbox=0)
#     number_concentration_diff = number_concentration_top - number_concentration_surface

#     mass_concentration = ds['mass_represented_temporal_mean'].sel(microphysics=microphysic).T
#     mass_concentration.attrs.update({"long_name": "Mass concentration", "units": "kg m^{-3}"})

#     mass_concentration_top = mass_concentration.sel(gridbox=ds["max_gridbox"].sel(microphysics=microphysic) - 1)
#     mass_concentration_ten = mass_concentration.sel(gridbox=10)
#     mass_concentration_surface = mass_concentration.sel(gridbox=0)
#     mass_concentration_diff = mass_concentration_top - mass_concentration_surface

#     num_conc = dict(
#         top = number_concentration_top,
#         ten = number_concentration_ten,
#         surface = number_concentration_surface,
#         diff = number_concentration_diff,
#     )

#     mass_conc = dict(
#         top = mass_concentration_top,
#         ten = mass_concentration_ten,
#         surface = mass_concentration_surface,
#         diff = mass_concentration_diff,
#     )

#     for d, axs in zip([num_conc, mass_conc], [axs_dsd, axs_msd]) :
#         for i, key in enumerate(d):
#             _y = d[key]
#             ax = axs[i]

#             ma, q25, q75 = _y.quantile([0.5, 0.25, 0.75], dim="cloud_id")
#             ax.plot(
#                 x,
#                 ma,
#                 color= dark_colors[microphysic],
#                 alpha = 1,
#                 linestyle = '-',
#                 label = labels[microphysic],
#                 # marker = '.'
#             );
#             ax.fill_between(
#                 x,
#                 q25,
#                 q75,
#                 color= colors[microphysic],
#                 alpha = 0.2,
#                 linestyle = '-',
#                 # label = labels[microphysic],
#                 # marker = '.'
#             );


#             # m, sem = mean_and_stderror_of_mean(_y, "cloud_id")
#             # ax.plot(
#             #     x,
#             #     m,
#             #     color= dark_colors[microphysic],
#             #     alpha = 1,
#             #     linestyle = '--',
#             #     label = labels[microphysic],
#             #     # marker = '.'
#             # );
#             # ax.fill_between(
#             #     x,
#             #     m + 2 * sem,
#             #     m - 2 * sem,
#             #     color= colors[microphysic],
#             #     alpha = 0.2,
#             #     linestyle = '-',
#             #     # label = labels[microphysic],
#             #     # marker = '.'
#             # );
#             ax.set_title(f"{key}")


# for ax in axs_dsd:
#     ax.set_xlabel(label_from_attrs(x))
#     ax.set_ylabel(label_from_attrs(number_concentration))
#     ax.set_xscale('log')
# for ax in axs_dsd[:-1]:
#     ax.set_ylim(0, 7e6)
# axs_dsd[-1].set_ylim(-2e6, 2e6)


# for ax in axs_msd:
#     ax.set_xlabel(label_from_attrs(x))
#     ax.set_ylabel(label_from_attrs(mass_concentration))
#     ax.set_xscale('log')
# for ax in axs_msd[:-1]:
#     ax.set_ylim(0, 1)
# axs_msd[-1].set_ylim(-0.1, 0.1)

# for ax in np.concatenate([axs_dsd, axs_msd]):
#     ax.set_xlim(20, None)
#     ax.legend(loc= 'upper left')

#     # axs[0, 0].set_yscale('log')

# fig.suptitle("All microphysics")

# fig.tight_layout()
# save_figure(fig, fig_dir / "all_microphysics_dsd_msd")

In [ ]:
# fig = plt.figure(layout='constrained', figsize=(12, 10))
# subfigs = fig.subfigures(nrows = 1, ncols = 2, wspace=0.07)

# axs_dsd = subfigs[0].subplots(nrows = 4, ncols = 1, sharey=False, sharex=True)
# axs_msd = subfigs[1].subplots(nrows = 4, ncols = 1, sharey=False, sharex=True)

# for microphysic in compare_microphysics:

#     x = ds_diff["radius_bins"]
#     x.attrs.update({"long_name": "Radius", "units": "µm"})
#     number_concentration = ds_diff['xi_temporal_mean'].sel(microphysics=microphysic).T
#     number_concentration.attrs.update({"long_name": "Number concentration", "units": "m^{-3}"})

#     number_concentration_top = number_concentration.sel(gridbox=ds["max_gridbox"].sel(microphysics=microphysic) - 1)
#     number_concentration_ten = number_concentration.sel(gridbox=10)
#     number_concentration_surface = number_concentration.sel(gridbox=0)
#     number_concentration_diff = number_concentration_top - number_concentration_surface

#     mass_concentration = ds_diff['mass_represented_temporal_mean'].sel(microphysics=microphysic).T
#     mass_concentration.attrs.update({"long_name": "Mass concentration", "units": "kg m^{-3}"})

#     mass_concentration_top = mass_concentration.sel(gridbox=ds["max_gridbox"].sel(microphysics=microphysic) - 1)
#     mass_concentration_ten = mass_concentration.sel(gridbox=10)
#     mass_concentration_surface = mass_concentration.sel(gridbox=0)
#     mass_concentration_diff = mass_concentration_top - mass_concentration_surface

#     num_conc = dict(
#         top = number_concentration_top,
#         ten = number_concentration_ten,
#         surface = number_concentration_surface,
#         diff = number_concentration_diff,
#     )

#     mass_conc = dict(
#         top = mass_concentration_top,
#         ten = mass_concentration_ten,
#         surface = mass_concentration_surface,
#         diff = mass_concentration_diff,
#     )

#     for d, axs in zip([num_conc, mass_conc], [axs_dsd, axs_msd]) :
#         for i, key in enumerate(d):
#             _y = d[key]
#             ax = axs[i]

#             ma, q25, q75 = _y.quantile([0.5, 0.25, 0.75], dim="cloud_id")
#             ax.plot(
#                 x,
#                 ma,
#                 color= dark_colors[microphysic],
#                 alpha = 1,
#                 linestyle = '-',
#                 label = labels[microphysic],
#                 # marker = '.'
#             );
#             ax.fill_between(
#                 x,
#                 q25,
#                 q75,
#                 color= dark_colors[microphysic],
#                 alpha = 0.2,
#                 linestyle = '-',
#                 # label = labels[microphysic],
#                 # marker = '.'
#             );


#             m, sem = mean_and_stderror_of_mean(_y, "cloud_id")
#             ax.plot(
#                 x,
#                 m,
#                 color= colors[microphysic],
#                 alpha = 1,
#                 linestyle = '--',
#                 label = labels[microphysic],
#                 # marker = '.'
#             );
#             ax.fill_between(
#                 x,
#                 m + 2 * sem,
#                 m - 2 * sem,
#                 color= colors[microphysic],
#                 alpha = 0.2,
#                 linestyle = '-',
#                 # label = labels[microphysic],
#                 # marker = '.'
#             );
#             ax.set_title(f"{key}")


# for ax in axs_dsd:
#     ax.set_xlabel(label_from_attrs(x))
#     ax.set_ylabel(label_from_attrs(number_concentration))
#     ax.set_xscale('log')
# for ax in axs_dsd[:-1]:
#     ax.set_ylim(0, 7e6)
# axs_dsd[-1].set_ylim(-2e6, 2e6)


# for ax in axs_msd:
#     ax.set_xlabel(label_from_attrs(x))
#     ax.set_ylabel(label_from_attrs(mass_concentration))
#     ax.set_xscale('log')
# for ax in axs_msd[:-1]:
#     ax.set_ylim(0, 1)
# axs_msd[-1].set_ylim(-0.1, 0.1)

# for ax in np.concatenate([axs_dsd, axs_msd]):
#     ax.set_xlim(20, None)
#     ax.legend(loc= 'upper left')

#     # axs[0, 0].set_yscale('log')

# fig.suptitle("All microphysics")

# fig.tight_layout()
# fig.tight_layout()
# save_figure(fig, fig_dir / "all_microphysics_dsd_msd-difference")

In [ ]:
# fig = plt.figure(layout='constrained', figsize=(12, 10))
# subfigs = fig.subfigures(nrows = 1, ncols = 2, wspace=0.07)

# axs_dsd = subfigs[0].subplots(nrows = 3, ncols = 1, sharey=True, sharex=True)
# axs_msd = subfigs[1].subplots(nrows = 3, ncols = 1, sharey=True, sharex=True)

# for microphysic in compare_microphysics:

#     x = ds["radius_bins"]
#     x.attrs.update({"long_name": "Radius", "units": "µm"})
#     y = ds_diff['xi_temporal_mean'].sel(microphysics=microphysic).T
#     y.attrs.update({"long_name": "Number concentration", "units": "m^{-3}"})

#     top = y.sel(gridbox=ds["max_gridbox"].sel(microphysics=microphysic) - 1)
#     surface = y.sel(gridbox=0)
#     diff = top - surface

#     for i, _y in enumerate([top, surface, diff]) :
#         # axs_dsd[i].plot(
#         #     x,
#         #     _y,
#         #     color= colors[microphysic],
#         #     alpha = 0.2,
#         #     linestyle = '-',
#         #     # marker = '.'
#         # );
#         axs_dsd[i].plot(
#             x,
#             _y.mean("cloud_id"),
#             color= dark_colors[microphysic],
#             alpha = 1,
#             linestyle = '-',
#             # marker = '.'
#             label = labels[microphysic],
#         );
#     axs_dsd[0].set_xscale('log')
#     axs_dsd[0].set_ylim(-4e6, 4e6)

#     for ax in axs_dsd:
#         ax.set_xlabel(label_from_attrs(x))
#         ax.set_ylabel(label_from_attrs(y))

#     x = ds["radius_bins"]
#     y = ds_diff['mass_represented_temporal_mean'].sel(microphysics=microphysic).T
#     y.attrs.update({"long_name": "Mass concentration", "units": "kg m^{-3}"})

#     top = y.sel(gridbox=ds["max_gridbox"].sel(microphysics=microphysic) - 1)
#     surface = y.sel(gridbox=0)
#     diff = top - surface

#     for i, _y in enumerate([top, surface, diff]) :
#         # axs_msd[i].plot(
#         #     x,
#         #     _y,
#         #     color= colors[microphysic],
#         #     alpha = 0.2,
#         #     linestyle = '-',
#         #     marker = 'None'
#         # );
#         axs_msd[i].plot(
#             x,
#             _y.mean("cloud_id"),
#             color= dark_colors[microphysic],
#             alpha = 1,
#             linestyle = '-',
#             label = labels[microphysic],
#             # marker = '.'
#         );


#     axs_msd[0].set_xscale('log')
#     axs_msd[0].set_ylim(-0.4, 0.4)

#     for ax in axs_msd:
#         ax.set_xlabel(label_from_attrs(x))
#         ax.set_ylabel(label_from_attrs(y))

# for ax in np.concatenate([axs_dsd, axs_msd]):
#     ax.set_xlim(20, None)
#     ax.legend(loc= 'upper left')

#     # axs[0, 0].set_yscale('log')

# fig.suptitle("Difference to condensation microphysics")

## Some more correlations

### Impoact of Rain water content

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = ds["cloud_liquid_water_content"]
y = -ds["source_energy"]

for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
# ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}_vs_{y.name}")

### Impact of relative humidity

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

x = (
    (ds["relative_humidity"].max("gridbox") - ds["relative_humidity"].min("gridbox"))
    / ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"])
    * 1e3
)
x.name = "RH_gradient"
x.attrs["units"] = r"\% km$^{-1}$"
x.attrs["long_name"] = "Relative Humidity Gradient"
y = -ds["source_energy"]  # / ds["gridbox_coord3"].sel(gridbox = ds['max_gridbox']) * 1e3


for mp in ["condensation"]:

    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        color=colors[mp],
        alpha=0.8,
        marker=markers[mp],
        label=labels[mp],
    )

ax.legend(loc="upper right")
# ax.set_yscale('log')
# ax.set_xscale("log")

ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))

fig.tight_layout()
save_figure(fig, fig_dir / f"{x.name}_vs_{y.name}")

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10, 10))


for i, (x, y) in enumerate(
    (
        (ds["relative_humidity"], -ds["evaporation_rate_energy"]),
        (ds["liquid_water_content"], -ds["evaporation_rate_energy"]),
        (ds["mass_radius_mean"], -ds["evaporation_rate_energy"]),
    )
):

    ax = axs[i, 0]
    ax_2 = axs[i, 1]
    da_corr = xr.corr(x, y, dim="gridbox")
    da_corr_2 = xr.corr(x, y, dim="gridbox")
    for mp in ["condensation"]:
        data = da_corr.sel(microphysics=mp)
        m, sem = mean_and_stderror_of_mean(data, dims="cloud_id")

        ax.hist(
            data,
            label=f"{labels[mp]}: {m.values:.2f} ± {sem.values:.2f}",
            color=colors[mp],
            alpha=0.2,
            bins=np.linspace(-1, 1, 20),
        )

        ax_2.scatter(
            y.sel(microphysics=mp),
            x.sel(microphysics=mp),
            color=colors[mp],
            alpha=0.5,
            marker=markers[mp],
            label=labels[mp],
        )

    ax.set_xlabel(label_from_attrs(x))
    ax.set_ylabel(label_from_attrs(y))
    ax_2.set_xlabel(label_from_attrs(y))
    ax_2.set_ylabel(label_from_attrs(x))
    ax.legend()

fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10, 10))


for i, (x, y) in enumerate(
    (
        (ds["relative_humidity"], -ds["evaporation_rate_energy"]),
        (ds["liquid_water_content"], -ds["evaporation_rate_energy"]),
        (ds["mass_radius_mean"], -ds["evaporation_rate_energy"]),
    )
):

    y_norm = (y / y.mean("gridbox")) * 100
    y_norm.attrs = y.attrs.copy()
    y_norm.attrs["units"] = "\\%"
    y_norm.attrs["long_name"] += " relative to column mean"

    ax: plt.Axes = axs[i, 0]
    ax_2: plt.Axes = axs[i, 1]
    da_corr = xr.corr(x, y_norm, dim="gridbox")

    for mp in ["condensation"]:
        data = da_corr.sel(microphysics=mp)
        m, sem = mean_and_stderror_of_mean(data, dims="cloud_id")

        ax.hist(
            data,
            label=f"{labels[mp]}: {m.values:.2f} ± {sem.values:.2f}",
            color=colors[mp],
            alpha=0.2,
            bins=np.linspace(-1, 1, 20),
        )

        ax_2.plot(
            y_norm.sel(microphysics=mp).T,
            x.sel(microphysics=mp).T,
            linewidth=0.5,
            alpha=0.5,
            color=colors[mp],
            # marker = markers[mp],
            label=labels[mp],
        )

    ax.set_xlabel(label_from_attrs(x))
    ax.set_ylabel(label_from_attrs(y))
    ax_2.set_xlabel(label_from_attrs(y_norm))
    ax_2.set_ylabel(label_from_attrs(x))
    ax_2.set_xlim(0, 250)
    ax.legend()

fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(5, 10))


for i, (x, y) in enumerate(
    (
        (ds["relative_humidity"], -ds["evaporation_rate_energy"]),
        (ds["liquid_water_content"], -ds["evaporation_rate_energy"]),
        (ds["mass_radius_mean"], -ds["evaporation_rate_energy"]),
    )
):
    ax = axs[i]
    da_corr = xr.corr(x, y, dim="cloud_id")
    for mp in microphysics:
        data = da_corr.sel(microphysics=mp)

        ax.scatter(
            data,
            data.gridbox,
            label=f"{labels[mp]}: {m.values:.2f} ± {sem.values:.2f}",
            color=colors[mp],
            alpha=0.2,
            # bins = np.linspace(-1, 1, 20)
        )

    ax.set_title(f"Correlation of {label_from_attrs(x)} and {label_from_attrs(y)}")
    ax.set_xlabel("Correlation")
    ax.set_ylabel("Gridbox")
    ax.legend()

fig.tight_layout()

In [ ]:
vars = (
    ds["relative_humidity"],
    ds["liquid_water_content"],
    ds["mass_radius_mean"],
    ds["evaporation_rate_energy"],
)

xx = vars
yy = vars

fig, axs = plt.subplots(len(xx), len(yy), figsize=(20, 20))


for i, y in enumerate(xx):
    for j, x in enumerate(yy):

        ax = axs[i, j]
        for mp in ["condensation"]:

            ax.scatter(
                x.sel(microphysics=mp),
                y.sel(microphysics=mp),
                color=colors[mp],
                alpha=0.8,
                marker=markers[mp],
                label=labels[mp],
            )

        # ax.legend(loc="upper right")
        # ax.set_yscale('symlog')
        # ax.set_xscale("symlog")

        ax.set_ylabel(label_from_attrs(y, name_width=10))
        ax.set_xlabel(label_from_attrs(x, name_width=10))

        # if j == 0:
        #     ax.set_ylabel(label_from_attrs(y))
        # if i == len(xx) - 1:
        #     ax.set_xlabel(label_from_attrs(x))


fig.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
import random
import matplotlib.ticker as mticker

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection="3d")


def log_tick_formatter(val, pos=None):
    return f"$10^{{{int(val)}}}$"


for mp in ["condensation"]:

    ds_sel = ds.sel(microphysics=mp)

    x = np.log10(ds_sel["liquid_water_content"])
    z = ds_sel["relative_humidity"]
    color = ds_sel["mass_radius_mean"]
    y = ds_sel["evaporation_rate_energy"]

    ax.scatter(
        x.values.flatten(),
        y.values.flatten(),
        z.values.flatten(),
        c=color.values.flatten(),
        alpha=0.5,
        cmap="inferno_r",
    )

    ax.plot(x.values, y.values, z.values, linewidth=0.5, color="k", alpha=1)

    ax.xaxis.set_major_formatter(mticker.FuncFormatter(log_tick_formatter))
    ax.xaxis.set_major_locator(mticker.MaxNLocator(integer=True))

In [ ]:
ds_surface = ds.sel(gridbox=0)
ds_cloud_base = ds.sel(gridbox=ds["max_gridbox"])

ds_slope = (
    (ds_cloud_base - ds_surface) / (ds_cloud_base["gridbox_coord3"] - ds_surface["gridbox_coord3"]) * 1e3
)

ds_relative = (ds / ds.mean("gridbox") - 1) * 100
ds_relative_surface = ds_relative.sel(gridbox=0)
ds_relative_cloud_base = ds_relative.sel(gridbox=ds["max_gridbox"])

ds_relative_slope = ds_relative_cloud_base - ds_relative_surface

for var in ds_slope.variables:
    attrs = ds[var].attrs.copy()
    relative_attrs = ds[var].attrs.copy()
    units_suffix = r" km^{-1}"
    long_name_suffix = " gradient"

    try:
        attrs["units"] += units_suffix
        relative_attrs["units"] = units_suffix
    except KeyError:
        print(f"No units for {var}")
    try:
        attrs["long_name"] += long_name_suffix
        relative_attrs["long_name"] += long_name_suffix
    except KeyError:
        print(f"No long name for {var}")

    ds_slope[var].attrs = attrs
    ds_relative_slope[var].attrs = relative_attrs

No units for cloud_id
No long name for cloud_id
No units for radius_bins
No long name for radius_bins
No units for microphysics
No long name for microphysics
No long name for air_temperature
No long name for mass_represented_temporal_mean
No long name for pressure
No long name for specific_mass_vapour
No long name for xi_temporal_mean


In [ ]:
vars = (
    ds_slope["relative_humidity"],
    ds_slope["liquid_water_content"],
    ds_slope["mass_radius_mean"],
    ds_slope["evaporation_rate_energy"],
    ds_relative_slope["evaporation_rate_energy"],
)
xx = vars
yy = vars

fig, axs = plt.subplots(len(xx), len(yy), figsize=(20, 20))


for i, y in enumerate(xx):
    for j, x in enumerate(yy):

        ax = axs[i, j]
        for mp in ["condensation"]:

            ax.scatter(
                x.sel(microphysics=mp),
                y.sel(microphysics=mp),
                color=colors[mp],
                alpha=0.8,
                marker=markers[mp],
                label=labels[mp],
            )

        # ax.legend(loc="upper right")
        # ax.set_yscale('symlog')
        # ax.set_xscale("symlog")

        ax.set_ylabel(label_from_attrs(y, name_width=10))
        ax.set_xlabel(label_from_attrs(x, name_width=10))

        # if j == 0:
        #     ax.set_ylabel(label_from_attrs(y))
        # if i == len(xx) - 1:
        #     ax.set_xlabel(label_from_attrs(x))


fig.tight_layout()

In [ ]:
vars = [
    ds_cloud_base["cloud_liquid_water_content"],
    ds_slope["relative_humidity"],
    ds_surface["relative_humidity"],
    -ds_cloud_base["source_energy"],
    -ds_slope["evaporation_rate_energy"],
    ds_relative_slope["evaporation_rate_energy"],
]

xx = vars
yy = vars

fig, axs = plt.subplots(len(xx), len(yy), figsize=(20, 20))


for i, y in enumerate(xx):
    for j, x in enumerate(yy):

        ax = axs[i, j]
        for mp in ["condensation"]:

            ax.scatter(
                x.sel(microphysics=mp),
                y.sel(microphysics=mp),
                color=colors[mp],
                alpha=0.8,
                marker=markers[mp],
                label=labels[mp],
            )

        # ax.legend(loc="upper right")
        # ax.set_yscale('symlog')
        # ax.set_xscale("symlog")

        ax.set_ylabel(label_from_attrs(y, name_width=10))
        ax.set_xlabel(label_from_attrs(x, name_width=10))

        # if j == 0:
        #     ax.set_ylabel(label_from_attrs(y))
        # if i == len(xx) - 1:
        #     ax.set_xlabel(label_from_attrs(x))


fig.tight_layout()

# Interpolation for vertical properties

In [ ]:
np.random.seed(2938)
cloud_ids = np.random.choice(ds["cloud_id"], 5, replace=False)

ds_sel = ds.isel(microphysics=2).sel(cloud_id=cloud_ids).sel(gridbox=[0, 10, 20, 30, 40, 50])
ds_sel = ds_sel.drop_vars("microphysics")


ds_example = xr.Dataset()
da = (ds_sel["massdelta_condensation"] * 0 + ds_sel["gridbox"]).compute()
norm_gridbox = ds_sel["gridbox"] / da.max("gridbox")

ds_example["data"] = da
ds_example["data"].attrs = dict(long_name="Example data", units="kg")
# ds_example['normalized_gridbox'] = xr.where(norm_gridbox <= 1, norm_gridbox, np.nan)
ds_example["normalized_gridbox"] = norm_gridbox
ds_example["normalized_gridbox"].attrs = dict(long_name="Normalized Gridbox", units="")
ds_example = ds_example.chunk({"cloud_id": -1})
ds_example["x"] = ("cloud_id", np.arange(len(ds_example["cloud_id"])))
ds_example = ds_example.swap_dims({"cloud_id": "x"}).drop_vars("cloud_id").transpose("gridbox", "x")
ds_example = ds_example.compute()
ds_example.attrs.update(
    name="example_data",
    description="Example data to test normalization",
)
ds_example.to_netcdf("example_data.nc")

In [ ]:
ds_example = xr.open_dataset("example_data.nc").compute()
display(ds_example)


fig, axs = plt.subplots(ncols=3, width_ratios=(1, 1, 0.1))
c = ds_example["data"]
x = ds_example["x"]

# Plot the data in physical coordinates
y = ds_example["gridbox"]
xx, yy = np.meshgrid(x, y)
sc = axs[0].scatter(xx, yy, c=c, s=100, cmap="inferno_r")
axs[0].set_xlabel(label_from_attrs(x, return_units=False))
axs[0].set_ylabel(label_from_attrs(y, return_units=False))
axs[0].set_title("Data in physical coordinates")


# Plot the data in normalized coordinates
y = ds_example["normalized_gridbox"]
xx, yy = x.expand_dims(gridbox=ds_example["gridbox"]), ds_example["normalized_gridbox"]
sc = axs[1].scatter(xx, yy, c=c, s=100, cmap="inferno_r")

axs[1].set_xlabel(label_from_attrs(x, return_units=False))
axs[1].set_ylabel(label_from_attrs(y, return_units=False))
axs[1].set_title("Data in normalized coordinates")

# Add colorbar
fig.colorbar(sc, cax=axs[2], label="Data")
fig.tight_layout()

In [ ]:
def interp(x, xp, fp, method="linear") -> np.ndarray:

    # print(x.shape, xp.shape, fp.shape)

    result = np.full_like(x, np.nan)

    mask = np.isfinite(xp) & np.isfinite(fp)
    indices = np.where(mask)[0]

    if np.sum(mask) > 2:

        xp = xp[mask]
        fp = fp[mask]

        if method == "linear":
            result = np.interp(x=x, xp=xp, fp=fp, left=np.nan, right=np.nan)
        elif method == "cubic":
            result = scipy.interpolate.CubicSpline(x=xp, y=fp, extrapolate=True)(x)
    else:
        result = np.full_like(x, np.nan)

    return result

In [ ]:
def normalize_by_coord(
    data: xr.DataArray,
    da_y: xr.DataArray,
    da_x: xr.DataArray,
    new_x_values: Union[np.ndarray, int, None] = None,
    new_y_values: Union[np.ndarray, int, None] = None,
    method: str = "linear",
    griddata_kwargs: dict = dict(),
) -> xr.DataArray:

    y_coords = da_y.coords
    y_set = set(y_coords)
    x_coords = da_x.coords
    x_set = set(x_coords)

    x_expand = dict([(key, da_y[key]) for key in y_set - x_set])
    y_expand = dict([(key, da_x[key]) for key in x_set - y_set])

    x = da_x.expand_dims(x_expand)
    y = da_y.expand_dims(y_expand)

    sorted_dims = list(data.dims)
    x = x.transpose(*sorted_dims)
    y = y.transpose(*sorted_dims)

    assert x.dims == y.dims == data.dims
    z_array = data.values.ravel()
    x_array = x.values.ravel()
    y_array = y.values.ravel()

    print(x_array.shape, y_array.shape, z_array.shape)

    mask = np.isfinite(z_array)
    x_array = x_array[mask]
    y_array = y_array[mask]
    z_array = z_array[mask]
    print(x_array.shape, y_array.shape, z_array.shape)

    # get coordiantes and data in vectorial form
    # interpolate unstructured grid data to structured x-Normalized Gridbox grid
    from scipy.interpolate import griddata

    if new_x_values is None:
        new_x = da_x.values
    elif isinstance(new_x_values, int):
        mini, maxi = da_x.min().values, da_x.max().values
        new_x = np.arange(mini, maxi, (maxi - mini) / new_x_values)
    elif isinstance(new_x_values, np.ndarray):
        new_x = new_x_values
    else:
        raise ValueError("new_x_values must be None, int or np.ndarray")

    if new_y_values is None:
        new_y = da_y.values
    elif isinstance(new_y_values, int):
        mini, maxi = da_y.min().values, da_y.max().values
        new_y = np.arange(mini, maxi, (maxi - mini) / new_y_values)
    elif isinstance(new_y_values, np.ndarray):
        new_y = new_y_values
    else:
        raise ValueError("new_y_values must be None, int or np.ndarray")

    new_xx, new_yy = np.meshgrid(new_x, new_y)

    new_z = griddata(
        (x_array, y_array), z_array.ravel(), (new_xx, new_yy), method=method, **griddata_kwargs
    )  # chose your favourite interpolation method.

    da_new_x = xr.DataArray(new_x, dims=[x.name], coords={f"{x.name}": new_x}, name=x.name)
    da_new_y = xr.DataArray(new_y, dims=[y.name], coords={f"{y.name}": new_y}, name=y.name)

    da_new = xr.DataArray(
        new_z.T,
        dims=[x.name, y.name],
        coords={y.name: da_new_y, x.name: da_new_x},
        name=data.name,
    )
    return da_new